# Imports/Init

In [3]:
%reload_ext autoreload
%autoreload 2
!mamba install tensorflow-gpu==2.10 -y -q

In [4]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import defaultdict
from joblib import Memory
import pickle
from pprint import pprint
from tqdm import tqdm

from typing import Dict

!pip install -q scikit-learn-intelex
#from sklearnex import patch_sklearn
#patch_sklearn()

from sklearn import pipeline, metrics, linear_model, model_selection, multioutput, tree, ensemble, neural_network
!pip install xgboost
import xgboost as xgb

import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.model_selection import train_test_split

import keras
from sklearn.utils import class_weight
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt 
from sklearn.metrics import multilabel_confusion_matrix, classification_report, confusion_matrix

from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')
gpus = tf.config.list_physical_devices('GPU') or []
for gpu in gpus:
    try:
        tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

  Using cached xgboost-1.7.3-py3-none-manylinux2014_x86_64.whl (193.6 MB)


2023-02-06 13:24:52.539796: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA A100 80GB PCIe MIG 3g.40gb, compute capability 8.0


In [5]:
memory = Memory(location='./cache')

In [25]:
from typing import Iterator, List, Tuple
global display_graph_counter
display_graph_counter=0



def appliance_augmentator(dataset:Dict[str,list], sample_length:int, n_appliances_per_sample:int, random_state:int=None) -> Iterator[Tuple[np.ndarray, np.ndarray, List[str]]]:
    global display_graph_counter
   
          
    # Initialize seeded random number generator
    rng = np.random.default_rng(seed=random_state)
    
    # How many appliances are mixed together
    N = n_appliances_per_sample
    
    # Sample length / Window size
    L = sample_length
    
    # Noise floor
    NOISE_FLOOR = 20.0
    
    # Get all available appliances
    appliance_names = tuple(dataset.keys())
    
    # How many appliances are there?
    n_appliances = len(appliance_names)
    #print(n_appliances)
    #print(N)
    # Start endless generator of samples
    # pre-allocate array for time series
    series = np.zeros(L, dtype=np.float64)
        
    # pre-allocate boolean array for masks
    labels = np.zeros((n_appliances, L), dtype=bool)

    if testing:
        print("test run")
        appliance_names=naprave

    print(appliance_names, " pri augmentarorju")
    

    # Select N random appliances (no replace, because same appliance should not appear twice in the same sample)
    # TODO: Relax this limitation in the future, for cases where multiple of same appliances are in the same household
    while True:
        for _ in tqdm(range(120000)):
            #print(_)
            series = np.zeros(L, dtype=np.float64)

            # pre-allocate boolean array for masks
            labels = np.zeros((n_appliances, L), dtype=bool)

            #print("///////////////////////////")
            for appliance_idx in rng.choice(n_appliances, size=N, replace=False):
                #print(appliance_names[appliance_idx])
                appliance_name = appliance_names[appliance_idx]
                
                # Pick random sample of a selected appliance
                n_available_samples = len(dataset[appliance_name])
                empty=True
                while empty:
                    n_available_samples = len(dataset[appliance_name])
                    sample_idx = rng.choice(n_available_samples)
                    #print("test1")
                    # retrieve sample as NumPy array with appropriate dimensions
 
                    sample_series = dataset[appliance_name][sample_idx].iloc[:].to_numpy().squeeze(axis=-1)
                        #with numpy.printoptions(threshold=numpy.inf):
                            #print(sample_series)            
                        # If sample is too short (shorter than L), give padding on both sides.
                    if len(sample_series) <= L:
                        padding = L // 2
                        sample_series = np.pad(sample_series, (padding, padding), mode='constant', constant_values=0)
                        #print("////////////////////////")
                        #with numpy.printoptions(threshold=numpy.inf):
                            #print(sample_series)
                        # The total length of sample time-series
                    sample_len = len(sample_series)

                        # Sanity check(s)
                    assert sample_len >= L, f'Sample length should be equal or larger than L: {sample_len} >= {L}'

                    sample_offset = rng.choice(sample_len - L)

                    sample = sample_series[sample_offset:sample_offset+L]

                        # TODO: Currently, we ignore device in idle state (x =< NOISE_FLOOR)
                    mask = sample > NOISE_FLOOR  # find samples that are above noise floor
                    if (np.any(mask)):
                        if(appliance_name=='electric oven'):
                            #print(sample_idx," :which sample")
                                
                            #plt.plot(sample)
                            #plt.title(appliance_name)
                            #plt.show()
                            pass
                            
                        series[:] += sample
                        labels[appliance_idx, :] |= mask
                        empty=False

                    # logical ORing the mask

                #print(np.sum([(np.any(label) > 0) for label in labels]), N)
                # Add random (constant) offset
                #series += rng.random() * (NOISE_FLOOR)

                # There has to be two samples present. Even though we combined two subsets, one or both could be empty.
                # Workaround until area of interest is implemented.
            #print(np.shape(series))
            #print(np.shape(labels))

            display_graph_counter+=1
            yield series, labels, appliance_names
            

In [6]:
import random
import numpy
import math
from sklearn.model_selection import train_test_split
from tqdm import tqdm
!pip install progress
from progress.bar import Bar

def DevicesDataXY(number_of_datasets: int, 
                  number_of_devices_in_datasets: int, 
                  number_of_all_devices: int):
    
    # This function uses processed_data and shapes it into two lists 
    # one of them contains data from devices (dataX_Y) and one of them names of devices (devicesX_Y).
    # Lists contain multiple datasets specified with number_of_datasets, all of which have 
    # the same number of devices in them, specified by number_of_devices_in_datasets.
    # We choose devices for datasets randomly, thus we have to specify the number of all devices in the REFIT (22) or UKDALE dataset (54).
    
    # we extract the dictionary processed_data into a list AllTable
    #print(processed_data.keys())
    #'boiler', 'solar thermal pumping station', 'laptop computer', 'washer dryer', 'dish washer'])
    #('laptop computer', 'solar thermal pumping station', 'boiler', 'dish washer', 'washer dryer')
    AllTable = [[k,v] for k,v in processed_data.items()]
    #print(AllTable[0][0])
    devicesX_Y = []
    dataX_Y = []
        

            
    # for loop goes over all datasets
    for i in range(0,number_of_datasets):    
        devicesY = []
        dataY = []
        j = 0
        
        # while loop goes over all devices in datasets
        while j < number_of_devices_in_datasets:
            
            # we get a random number with random library
            random_number = random.randrange(number_of_all_devices)
            
            # we use the random number to get a random device and random data that belongs to it from AllTable
            random_device = AllTable[random_number][0]
            random_data = AllTable[random_number][1]
            
            # we append the device and its data if it doesn't already exist 
            # and thus avoid having the same device more then once in the same dataset
            if random_device not in devicesY:
                devicesY.append(random_device)
                dataY.append(random_data)
                j += 1
        
        devicesX_Y.append(devicesY)
        dataX_Y.append(dataY)
    
    return devicesX_Y, dataX_Y


def ListsToDictlist(devices: list, data: list, number_of_devices: int):
    
    # This function takes lists from DevicesDataXY or DevicesDataHoly5 and
    # turns each of the sublists into a dictionary and then appends those dicitonaries into a list. 
    
    Dictlist=[]
    
    # we make dictionaries out of lists that we input and append them to list of dictionaries (Dictlist)
    for i in range(0,number_of_devices):
        dictionary=dict(zip(devices[i], data[i]))
        Dictlist.append(dictionary)
        
    return Dictlist

def Generate4Upgraded(Dictlist: list, 
              sample_length: int, 
              n_appliances_per_sample: int, 
              dataset_number: int,
              number_of_generated_samples: int
                     ):
    
    # This function does the same as Generate4 except here we can specify the number of all generated samples. 
    
    X, Y, labels = [], [], None
    
    generator = appliance_augmentator(Dictlist[dataset_number], 
                                      sample_length=sample_length, 
                                      n_appliances_per_sample=n_appliances_per_sample,
                                      #n_samples=number_of_generated_samples,
                                      random_state=0xDEADBEEF)
    for idx, (_x, _y, labels) in zip((range(number_of_generated_samples)), generator):
        X.append(_x), Y.append(_y)
  
    X, Y = np.asarray(X), np.asarray(Y)
    y = np.any(Y, axis=-1) > 0
    sample_weight = np.sum(Y, axis=-1) / 128
    
    # split into train test sets
    from sklearn.model_selection import train_test_split
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    # add the needed dimension
    x_train = np.expand_dims(x_train, -1)
    x_test = np.expand_dims(x_test, -1)
    
    return x_train, x_test, y_train, y_test, labels


def class_weights_tool(y_test):
    
    # This function returns class weights for each device in a prticular dataset in a form of a dicitionary.
    # It does so by simply counting how many times the device is present througout the dataset.
    
    # inspired by ronnie coleman
    light_weight, nums = [], []
    
    # this for loop goes over the collumns of the y_test dataset
    for j in range(0,len(y_test[0])):             
        
        count=0
        
        # gives 0,1,2,3,4,5,6....
        nums.append(j)
        
        # this loop goes over the rows in the y_test dataset
        for i in range(0,len(y_test)):
            
            # we count Trues in the whole column of y_test dataset
            if y_test[i][j] == True: count+=1     
        
        # we append Trues for the column of y_test dataset to the list light_weight
        light_weight.append(count)
        
    # makes the dictionary    
    class_weights_dictionary = dict(zip(nums, light_weight))             
    
    return class_weights_dictionary

  Using cached progress-1.6-py3-none-any.whl


# Models

In [7]:
import numpy as np
import warnings
import math
import pickle
import pandas as pd
from joblib import Memory
import matplotlib.pyplot as plt

from keras.models import Model
from keras.layers import Flatten, Dense, Input, GRU, BatchNormalization, LSTM, Bidirectional, AveragePooling1D
from keras.layers import Conv1D, Conv1DTranspose, LocallyConnected1D, SeparableConv1D, ConvLSTM1D
from keras.layers import MaxPooling1D, Dropout
from keras.layers import GlobalMaxPooling1D
from keras.layers import GlobalAveragePooling1D
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
#from keras.applications.imagenet_utils import _obtain_input_shape
#from keras.engine.topology import get_source_inputs

def LOL2(classes,
             window_size,
             method,
             method_num,
             include_top=True,
             input_tensor=None,
              pooling=None):

    # Determine proper input shape
    input_shape = (window_size,1)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
    # Block 1
    x = Conv1D(64, (3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv1D(64, (3), activation='relu', padding='same', name='block1_conv2')(x)
    x = AveragePooling1D((2), strides=(2), name='block1_pool')(x)

    # Block 2
    x = Conv1D(128, (3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv1D(128, (3), activation='relu', padding='same', name='block2_conv2')(x)
    x = AveragePooling1D((2), strides=(2), name='block2_pool')(x)

    # Block 3
    x = Conv1D(256, (3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv1D(256, (3), activation='relu', padding='same', name='block3_conv2')(x)
    x = AveragePooling1D((2), strides=(2), name='block3_pool')(x)

    # Block 4
    x = Conv1D(512, (3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv1D(512, (3), activation='relu', padding='same', name='block4_conv2')(x)
    x = AveragePooling1D((2), strides=(2), name='block4_pool')(x)
    
    # Block 5
    x = Conv1DTranspose(512, (3), activation='relu', padding='same', name='block5_tran_conv1')(x)
    #x = Conv1DTranspose(512, (3), activation='relu', padding='same', name='block5_tran_conv2')(x)
    x = AveragePooling1D((2), strides=(2), name='block5_pool')(x)
    
    # GRU layer, btw GRU stands for Gated Recurrent Units; https://en.wikipedia.org/wiki/Gated_recurrent_unit
    if method == 'gru':
        print("GRU enabled")
        x = GRU(method_num, activation='tanh', recurrent_activation='sigmoid')(x)                
        
    if method == 'gru2':
        print("GRU enabled")
        x = GRU(method_num,activation='tanh',recurrent_activation='sigmoid',reset_after=True)(x)        
        
    if method == 'bigru':
        print("Bi-GRU enabled")
        x = Bidirectional(GRU(method_num, activation='tanh', recurrent_activation='sigmoid'))(x)
        
    if method == 'lstm':
        print("LSTM enabled")
        x = LSTM(method_num, activation='tanh', recurrent_activation='sigmoid')(x)
    
    # LSTM layer, btw LSTM stands for long short-term memory; https://en.wikipedia.org/wiki/Long_short-term_memory
    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dense(4096, activation='relu', name='fc2')(x)
        x = Dense(classes, activation='sigmoid', name='predictions')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling1D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling1D()(x)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input
    # Create model.
    model = Model(inputs, x, name='LOL2')
    return model

In [2]:
def Generate4Upgraded(Dictlist: list, 
              sample_length: int, 
              n_appliances_per_sample: int, 
              dataset_number: int,
              number_of_generated_samples: int
                     ):
    
    # This function does the same as Generate4 except here we can specify the number of all generated samples. 
    
    X, Y, labels = [], [], None
    
    generator = appliance_augmentator(Dictlist[dataset_number], 
                                      sample_length=sample_length, 
                                      n_appliances_per_sample=n_appliances_per_sample,
                                      #n_samples=number_of_generated_samples,
                                      random_state=0xDEADBEEF)
    for idx, (_x, _y, labels) in zip((range(number_of_generated_samples)), generator):
        X.append(_x), Y.append(_y)
  
    X, Y = np.asarray(X), np.asarray(Y)
    y = np.any(Y, axis=-1) > 0
    sample_weight = np.sum(Y, axis=-1) / 128
    
    # split into train test sets
    from sklearn.model_selection import train_test_split
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    # add the needed dimension
    x_train = np.expand_dims(x_train, -1)
    x_test = np.expand_dims(x_test, -1)
    
    return x_train, x_test, y_train, y_test, labels


def DevicesDataXY(number_of_datasets: int, 
                  number_of_devices_in_datasets: int, 
                  number_of_all_devices: int):
    
    # This function uses processed_data and shapes it into two lists 
    # one of them contains data from devices (dataX_Y) and one of them names of devices (devicesX_Y).
    # Lists contain multiple datasets specified with number_of_datasets, all of which have 
    # the same number of devices in them, specified by number_of_devices_in_datasets.
    # We choose devices for datasets randomly, thus we have to specify the number of all devices in the REFIT (22) or UKDALE dataset (54).
    
    # we extract the dictionary processed_data into a list AllTable
    #print(processed_data.keys())
    #'boiler', 'solar thermal pumping station', 'laptop computer', 'washer dryer', 'dish washer'])
    #('laptop computer', 'solar thermal pumping station', 'boiler', 'dish washer', 'washer dryer')
    AllTable = [[k,v] for k,v in processed_data.items()]
    #print(AllTable[0][0])
    devicesX_Y = []
    dataX_Y = []
        

            
    # for loop goes over all datasets
    for i in range(0,number_of_datasets):    
        devicesY = []
        dataY = []
        j = 0
        
        # while loop goes over all devices in datasets
        while j < number_of_devices_in_datasets:
            
            # we get a random number with random library
            random_number = random.randrange(number_of_all_devices)
            
            # we use the random number to get a random device and random data that belongs to it from AllTable
            random_device = AllTable[random_number][0]
            random_data = AllTable[random_number][1]
            
            # we append the device and its data if it doesn't already exist 
            # and thus avoid having the same device more then once in the same dataset
            if random_device not in devicesY:
                devicesY.append(random_device)
                dataY.append(random_data)
                j += 1
        
        devicesX_Y.append(devicesY)
        dataX_Y.append(dataY)
    
    return devicesX_Y, dataX_Y

def ListsToDictlist(devices: list, data: list, number_of_devices: int):
    
    # This function takes lists from DevicesDataXY or DevicesDataHoly5 and
    # turns each of the sublists into a dictionary and then appends those dicitonaries into a list. 
    
    Dictlist=[]
    
    # we make dictionaries out of lists that we input and append them to list of dictionaries (Dictlist)
    for i in range(0,number_of_devices):
        dictionary=dict(zip(devices[i], data[i]))
        Dictlist.append(dictionary)
        
    return Dictlist
def class_weights_tool(y_test):
    
    # This function returns class weights for each device in a prticular dataset in a form of a dicitionary.
    # It does so by simply counting how many times the device is present througout the dataset.
    
    # inspired by ronnie coleman
    light_weight, nums = [], []
    
    # this for loop goes over the collumns of the y_test dataset
    for j in range(0,len(y_test[0])):             
        
        count=0
        
        # gives 0,1,2,3,4,5,6....
        nums.append(j)
        
        # this loop goes over the rows in the y_test dataset
        for i in range(0,len(y_test)):
            
            # we count Trues in the whole column of y_test dataset
            if y_test[i][j] == True: count+=1     
        
        # we append Trues for the column of y_test dataset to the list light_weight
        light_weight.append(count)
        
    # makes the dictionary    
    class_weights_dictionary = dict(zip(nums, light_weight))             
    
    return class_weights_dictionary

In [ ]:
from typing import Iterator, List, Tuple
global display_graph_counter
display_graph_counter=0



def appliance_augmentator(dataset:Dict[str,list], sample_length:int, n_appliances_per_sample:int, random_state:int=None) -> Iterator[Tuple[np.ndarray, np.ndarray, List[str]]]:
    global display_graph_counter
   
    # Initialize seeded random number generator
    rng = np.random.default_rng(seed=random_state)
    
    # How many appliances are mixed together
    N = n_appliances_per_sample
    
    # Sample length / Window size
    L = sample_length
    
    # Noise floor
    NOISE_FLOOR = 20.0
    
    # Get all available appliances
    appliance_names = tuple(dataset.keys())
    
    # How many appliances are there?
    n_appliances = len(appliance_names)

    # Start endless generator of samples
    # pre-allocate array for time series
    series = np.zeros(L, dtype=np.float64)
        
    # pre-allocate boolean array for masks
    labels = np.zeros((n_appliances, L), dtype=bool)

    if testing:
        print("test run")
        appliance_names=naprave

    print(appliance_names, " pri augmentarorju")
    

    # Select N random appliances (no replace, because same appliance should not appear twice in the same sample)
    # TODO: Relax this limitation in the future, for cases where multiple of same appliances are in the same household
    while True:
        for _ in tqdm(range(120000)):
            #print(_)
            series = np.zeros(L, dtype=np.float64)

            # pre-allocate boolean array for masks
            labels = np.zeros((n_appliances, L), dtype=bool)

            #print("///////////////////////////")
            for appliance_idx in rng.choice(n_appliances, size=N, replace=False):
                #print(appliance_names[appliance_idx])
                appliance_name = appliance_names[appliance_idx]
                
                # Pick random sample of a selected appliance
                n_available_samples = len(dataset[appliance_name])
                empty=True
                while empty:
                    n_available_samples = len(dataset[appliance_name])
                    sample_idx = rng.choice(n_available_samples)
                    #print("test1")
                    # retrieve sample as NumPy array with appropriate dimensions
 
                    sample_series = dataset[appliance_name][sample_idx].iloc[:].to_numpy().squeeze(axis=-1)
                        #with numpy.printoptions(threshold=numpy.inf):
                            #print(sample_series)            
                        # If sample is too short (shorter than L), give padding on both sides.
                    if len(sample_series) <= L:
                        padding = L // 2
                        sample_series = np.pad(sample_series, (padding, padding), mode='constant', constant_values=0)
                        #print("////////////////////////")
                        #with numpy.printoptions(threshold=numpy.inf):
                            #print(sample_series)
                        # The total length of sample time-series
                    sample_len = len(sample_series)

                    assert sample_len >= L, f'Sample length should be equal or larger than L: {sample_len} >= {L}'

                    sample_offset = rng.choice(sample_len - L)

                    sample = sample_series[sample_offset:sample_offset+L]

                    mask = sample > NOISE_FLOOR  # find samples that are above noise floor
                    if (np.any(mask)):                   
                        series[:] += sample
                        labels[appliance_idx, :] |= mask
                        empty=False

                    # logical ORing the mask


            display_graph_counter+=1
            yield series, labels, appliance_names
            

# Testing

In [5]:
# Dict to give us a list of devices to train with
with open('HES_processed', 'rb') as handle:
    needed_devices = pickle.load(handle)
# Dict with which we will train the model    
with open('5merged.pickle', 'rb') as handle:
    processed_data = pickle.load(handle)

# create a dict with only the required (shared) devices
processed_data = {k: v for k, v in processed_data.items() if k in needed_devices}
print(len(processed_data.keys()))

5


In [79]:
from keras import backend as K
import gc
K.clear_session()
gc.collect()
date = "28.9.2022"
net_name = 'LOL2'
window_size= 2550
batch_size = 512
NmSets = 3
learning_rate = 0.001
testing=False

#devicesX_Y, dataX_Y = DevicesDataXY(NmSets,NmDevices,54)

logical_gpus = tf.config.list_logical_devices('GPU')
print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
for tačka in range(0,2):
    mačka = [2,3]
    options = mačka[tačka]
    for turtle in range(0,1):
        beno = [5]
        NmDevices = beno[turtle]
        devicesX_Y, dataX_Y = DevicesDataXY(NmSets,NmDevices,5) # last parameter is 22 for refit and 54 for ukdale
        
        Dictlist = ListsToDictlist(devicesX_Y, dataX_Y, NmSets)
        if options < NmDevices: 
            for nm in range(2,NmSets):
                # |||||||||||||||||||||||||||||||| parameters ||||||||||||||||||||||||||||||||
                print("number of AD/DiT: ", options, " / ", NmDevices)
                print("repetition number: ", nm+1, "/", NmSets)
                if options == 1: 
                    epochs = 30
                    learning_rate = 0.0001
                elif options == 2: 
                    epochs = 25
                    learning_rate = 0.0001                              
                else: 
                    epochs = 20    
                    learning_rate = 0.0003
                    #learning_rate = 0.0003 # dej 0.0003 kot default za LOL2 (ecopirnat) (0,0001 za vgg)
                batch_size=128
                # |||||||||||||||||||||||||||||||| dataengineering ||||||||||||||||||||||||||||||||
                x_train, x_test, y_train, y_test, labels = Generate4Upgraded(Dictlist, window_size, options, nm, 120000)

                pickle.dump(labels, open(f'./results_5merged_HES_oven1/naprave/devices_({nm+1})_{epochs}ep_bs{batch_size}_W{window_size}_Devices{NmDevices}_opt{options}.pkl', 'wb'))
                class_weights = class_weights_tool(y_test)
                # |||||||||||||||||||||||||||||||| DL ||||||||||||||||||||||||||||||||
                model = LOL2(NmDevices, window_size, 'gru', 64)
                #model = VGG11_1D(NmDevices, window_size)
                #model = Tanoni()
                model.build((len(y_train)+len(y_test),window_size,1))
                model.summary()
                
                model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),loss='binary_crossentropy',metrics=['accuracy'])
                model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, class_weight=class_weights)
                

                model.save(f'./results_5merged_HES_oven1/modeli_fixed/({nm+1})_{epochs}ep_bs{batch_size}_W{window_size}_Devices{NmDevices}_opt{options}/')
                #model.save(f'./models/rawTS/LOL2/(1)_20ep_bs128_W2550_Devices10_opt9/')
                SaveData = [x_train, x_test, y_train, y_test, labels]
                #pickle.dump(SaveData, open(f'./results_5merged_HES/dataseti/dataset_({nm+1})_{epochs}ep_bs{batch_size}_W{window_size}_Devices{NmDevices}_opt{options}.pkl', 'wb'))
                # |||||||||||||||||||||||||||||||| prediction boiler||||||||||||||||||||||||||||||||
                y_pred = model.predict(x_test)
                y_pred_tf = (y_pred > 0.5)
                print(f"||||||||||||||||||||||||||||||||||||| {nm} |||||||||||||||||||||||||||||||||||||")
                print(metrics.classification_report(y_test, y_pred_tf, target_names=labels, zero_division=0))
                F1weighted = metrics.f1_score(y_test, y_pred_tf, labels=None, pos_label=1, average='weighted', sample_weight=None, zero_division=0)
                pickle.dump(F1weighted, open(f'./results_5merged_HES/rezultati/({nm+1})_F1Weighted_NmDevices{NmDevices}_opt{options}','wb'))
                print("|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||")
                

1 Physical GPUs, 1 Logical GPUs
number of AD/DiT:  2  /  5
repetition number:  3 / 3
('fridge', 'washing machine', 'air conditioner', 'microwave', 'electric oven')  pri augmentarorju


100%|█████████▉| 119999/120000 [00:32<00:00, 3720.06it/s]


GRU enabled
Model: "LOL2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2550, 1)]         0         
                                                                 
 block1_conv1 (Conv1D)       (None, 2550, 64)          256       
                                                                 
 block1_conv2 (Conv1D)       (None, 2550, 64)          12352     
                                                                 
 block1_pool (AveragePooling  (None, 1275, 64)         0         
 1D)                                                             
                                                                 
 block2_conv1 (Conv1D)       (None, 1275, 128)         24704     
                                                                 
 block2_conv2 (Conv1D)       (None, 1275, 128)         49280     
                                                  

INFO:tensorflow:Assets written to: ./results_5merged_HES_oven1/modeli_fixed/(3)_25ep_bs128_W2550_Devices5_opt2/assets


INFO:tensorflow:Assets written to: ./results_5merged_HES_oven1/modeli_fixed/(3)_25ep_bs128_W2550_Devices5_opt2/assets


...

100%|█████████▉| 119999/120000 [00:43<00:00, 2740.32it/s]


GRU enabled
Model: "LOL2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 2550, 1)]         0         
                                                                 
 block1_conv1 (Conv1D)       (None, 2550, 64)          256       
                                                                 
 block1_conv2 (Conv1D)       (None, 2550, 64)          12352     
                                                                 
 block1_pool (AveragePooling  (None, 1275, 64)         0         
 1D)                                                             
                                                                 
 block2_conv1 (Conv1D)       (None, 1275, 128)         24704     
                                                                 
 block2_conv2 (Conv1D)       (None, 1275, 128)         49280     
                                                  

INFO:tensorflow:Assets written to: ./results_5merged_HES_oven1/modeli_fixed/(3)_20ep_bs128_W2550_Devices5_opt3/assets


INFO:tensorflow:Assets written to: ./results_5merged_HES_oven1/modeli_fixed/(3)_20ep_bs128_W2550_Devices5_opt3/assets


750/750 [==============================] - 3s 4ms/step
||||||||||||||||||||||||||||||||||||| 2 |||||||||||||||||||||||||||||||||||||
                 precision    recall  f1-score   support

  electric oven       0.97      0.97      0.97     14358
         fridge       0.91      0.89      0.90     14391
washing machine       0.92      0.91      0.91     14408
      microwave       0.92      0.85      0.88     14408
air conditioner       0.99      0.98      0.98     14435

      micro avg       0.94      0.92      0.93     72000
      macro avg       0.94      0.92      0.93     72000
   weighted avg       0.94      0.92      0.93     72000
    samples avg       0.95      0.92      0.93     72000

|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||


# Validation

In [254]:
import pickle
import keras
def avg(lst):
    return sum(lst)/len(lst)
global display_graph_counter
display_graph_counter=0
iteration = 13
date = "28.9.2022"
epochs = 20
net_name = 'VGG11_1D'
window_size= 2550
batch_size = 128
NmSets = 2
NmDevices = 5
learning_rate = 0.001

testing=True
    

print("---------------------------------------------------------------------------")
for tačka in range(0,6):
    mačka = [4]
    options = mačka[tačka]
    for turtle in range(0,3):
        beno = [5]
        NmDevices = beno[turtle]
        #NmDevices = beno[turtle]
        #devicesX_Y, dataX_Y = DevicesDataXY(NmSets,NmDevices,54) # last parameter is 22 for refit and 54 for ukdale
        #Dictlist = ListsToDictlist(devicesX_Y, dataX_Y, NmSets)
        if options < NmDevices: 
            for nm in range(1,NmSets):
                # |||||||||||||||||||||||||||||||| parameters ||||||||||||||||||||||||||||||||
                
                print("number of AD/DiT: ", options, " / ", NmDevices)
                print("repetition number: ", nm+1, "/", NmSets)
                
                data = pickle.load(open('results_5merged_HES_oven1/naprave/devices_(3)_20ep_bs512_W2550_Devices5_opt4.pkl', 'rb'))
                naprave=data
                print(naprave, "original dataset")
                
                
                processed_data_copy=processed_data.copy()
                for k, v in processed_data_copy.items():
                    if k not in naprave:
                        del processed_data[k]
                        
                print("po spremembi", processed_data.keys())
                
                devicesX_Y, dataX_Y = DevicesDataXY(NmSets,NmDevices,5) # last parameter is 22 for refit and 54 for ukdale
                Dictlist = ListsToDictlist(devicesX_Y, dataX_Y, NmSets)
                x_train, x_test, y_train, y_test, labels = Generate4Upgraded(Dictlist, window_size, options, nm, 120000)
                data = [x_train, x_test, y_train, y_test, labels]
                class_weights = class_weights_tool(y_test) 
 
                saved_model = keras.models.load_model('results_5merged_HES_oven1/modeli_fixed/(3)_20ep_bs128_W2550_Devices5_opt4')
                y_pred = saved_model.predict(data[1])
        
                y_pred_tf = (y_pred > 0.5)
                #print(y_pred_tf)
                report = metrics.classification_report(data[3], y_pred_tf, target_names=data[4], zero_division=0)
                f1score = metrics.f1_score(data[3], y_pred_tf, labels=None, pos_label=1, average='weighted', sample_weight=None, zero_division=0)
                
                print(metrics.classification_report(y_test, y_pred_tf, target_names=labels, zero_division=0))

                print(f"Accuracy with  {NmDevices} devices and {options} options is: ", f1score)

            
        else: 
            results[turtle][tačka].append(0)
        FinalResults[turtle].append(round(avg(results[turtle][tačka])*100,2))
        
print(FinalResults)
pickle.dump(FinalResults, open(f'./datasets/rawTS/| LOL2_DALE |/FinalResults_2.pkl', 'wb'))

|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
---------------------------------------------------------------------------
number of AD/DiT:  4  /  5
repetition number:  2 / 2
('microwave', 'washing machine', 'air conditioner', 'fridge', 'electric oven') original dataset
po spremembi dict_keys(['microwave', 'fridge', 'washing machine', 'electric oven', 'air conditioner'])
test run
('microwave', 'washing machine', 'air conditioner', 'fridge', 'electric oven')  pri augmentarorju


100%|█████████▉| 119999/120000 [00:50<00:00, 2392.67it/s]


[[ True  True  True  True False]
 [False  True  True  True  True]
 [ True  True  True False  True]
 ...
 [ True False  True  True  True]
 [ True  True  True  True False]
 [ True  True False  True  True]]
750/750 [==============================] - 3s 4ms/step
                 precision    recall  f1-score   support

      microwave       0.81      0.97      0.88     19142
washing machine       0.82      0.88      0.85     19235
air conditioner       0.92      0.94      0.93     19273
         fridge       0.82      0.93      0.87     19089
  electric oven       0.81      0.90      0.86     19261

      micro avg       0.83      0.92      0.88     96000
      macro avg       0.84      0.92      0.88     96000
   weighted avg       0.84      0.92      0.88     96000
    samples avg       0.84      0.92      0.88     96000

Accuracy with  5 devices and 4 options is:  0.8768019793984421


IndexError: list index out of range